In [3]:
# unbalanced structure
import pandas as pd

case1 = 41
case2 = 42

bulkload_sizes = [0, 1000000, 2000000, 5000000, 10000000, 20000000, 50000000, 100000000]
# bulkload_ratios = ["0.0", "0.005", "0.01", "0.025", "0.05", "0.1", "0.25", "0.5"]
datasets = ["datasets/linear", "datasets/covid", "datasets/fb-1", "datasets/osm"]
index = "lipp"

df = pd.read_csv("../data/AIO_trimed.csv")
df = df[(df["read_ratio"] != 0) & (df["index_type"] == index)]
df['key_path'] = df['key_path'].apply(lambda x: x.replace('fb', 'fb-1'))
df_grouped = df.groupby(['index_type', 'key_path', 'test_suite', 'init_table_size'])['throughput'].mean().reset_index()
# print(len(df_grouped))
df_grouped.to_csv("grouped.csv", index=False)

# print(df_grouped)


# for each bulkload sizes, compute case2 and case1 throughput change ratio
for dataset in datasets:
    for bulkload_size in bulkload_sizes:
        # get case1 throughput
        d1 = df_grouped[(df_grouped["key_path"] == dataset) & (df_grouped["init_table_size"] == bulkload_size) & (df_grouped["test_suite"] == case1) & (df_grouped["index_type"] == index)]["throughput"].values
        d2 = df_grouped[(df_grouped["key_path"] == dataset) & (df_grouped["init_table_size"] == bulkload_size) & (df_grouped["test_suite"] == case2) & (df_grouped["index_type"] == index)]["throughput"].values
        print(f"dataset: {dataset}, bulkload_size: {bulkload_size}, {case1}: {d1}, {case2}: {d2}, ratio: {d2/d1}")



dataset: datasets/linear, bulkload_size: 0, 41: [], 42: [], ratio: []
dataset: datasets/linear, bulkload_size: 1000000, 41: [690663.5], 42: [1909982.5], ratio: [2.76543136]
dataset: datasets/linear, bulkload_size: 2000000, 41: [687043.], 42: [4506337.], ratio: [6.55903197]
dataset: datasets/linear, bulkload_size: 5000000, 41: [711764.], 42: [4521134.5], ratio: [6.35201345]
dataset: datasets/linear, bulkload_size: 10000000, 41: [749483.5], 42: [4665666.], ratio: [6.22517507]
dataset: datasets/linear, bulkload_size: 20000000, 41: [842579.5], 42: [4730053.5], ratio: [5.6137771]
dataset: datasets/linear, bulkload_size: 50000000, 41: [1201801.5], 42: [5517934.], ratio: [4.59138552]
dataset: datasets/linear, bulkload_size: 100000000, 41: [2470551.5], 42: [3012352.5], ratio: [1.21930367]
dataset: datasets/covid, bulkload_size: 0, 41: [], 42: [], ratio: []
dataset: datasets/covid, bulkload_size: 1000000, 41: [511436.5], 42: [1956055.], ratio: [3.82462926]
dataset: datasets/covid, bulkload_size

In [11]:
# overfit <pair>
import pandas as pd

# case = 21
# index = "alex"

bulkload_sizes = [0, 1000000, 2000000, 5000000, 10000000, 20000000, 50000000, 100000000]
sizes2ratios = {
    0: "0.0",
    1000000: "0.005",
    2000000: "0.01",
    5000000: "0.025",
    10000000: "0.05",
    20000000: "0.1",
    50000000: "0.25",
    100000000: "0.5",
}
# bulkload_ratios = ["0.0", "0.005", "0.01", "0.025", "0.05", "0.1", "0.25", "0.5"]
datasets = ["linear", "covid", "fb-1", "osm"]

df = pd.read_csv("../data/AIO_trimed.csv")
df = df[(df["read_ratio"] != 0)]
df['key_path'] = df['key_path'].apply(lambda x: x.replace('fb', 'fb-1'))
df_grouped = df.groupby(['index_type', 'key_path', 'test_suite', 'init_table_size'])['throughput'].mean().reset_index()
# print(len(df_grouped))
df_grouped.to_csv("grouped.csv", index=False)

# print(df_grouped)

for index in ["alex", "lipp"]:
    for case in [21, 22, 41, 42]:
        # for each bulkload sizes
        for dataset in datasets:
            for i in range(len(bulkload_sizes) - 1):
                for j in range(i+1, len(bulkload_sizes)):
                    # get case throughput
                    d1 = df_grouped[(df_grouped["key_path"] == f"datasets/{dataset}") & (df_grouped["init_table_size"] == bulkload_sizes[i]) & (df_grouped["test_suite"] == case) & (df_grouped["index_type"] == index)]["throughput"].values
                    d2 = df_grouped[(df_grouped["key_path"] == f"datasets/{dataset}") & (df_grouped["init_table_size"] == bulkload_sizes[j]) & (df_grouped["test_suite"] == case) & (df_grouped["index_type"] == index)]["throughput"].values
                    if (case == 41 or case == 42) and bulkload_sizes[i] == 0:
                        continue
                    root_model_file1 = f"../log/root_smo_profiling/{dataset}/{case}/{sizes2ratios[bulkload_sizes[i]]}/{index}_insert_root.log"
                    root_model_file2 = f"../log/root_smo_profiling/{dataset}/{case}/{sizes2ratios[bulkload_sizes[j]]}/{index}_insert_root.log"
                    p1 = pd.read_csv(root_model_file1)
                    p2 = pd.read_csv(root_model_file2)
                    is_model_change = False if len(p1) == 1 and len(p2) == 1 else True
                    if is_model_change == False and d1[0] > d2[0] and d1[0] / d2[0] > 1.2:
                        print(f"index: {index}, case:{case}, dataset:{dataset}, bulkload_size: {bulkload_sizes[i]}/{bulkload_sizes[j]}, {d1}/{d2}={d1/d2}, model_change: {is_model_change}")



index: alex, case:21, dataset:linear, bulkload_size: 0/1000000, [12233108.]/[4407167.5]=[2.77573022], model_change: False
index: alex, case:21, dataset:linear, bulkload_size: 0/2000000, [12233108.]/[3851564.5]=[3.17613998], model_change: False
index: alex, case:21, dataset:linear, bulkload_size: 0/5000000, [12233108.]/[3925004.5]=[3.11671184], model_change: False
index: alex, case:21, dataset:linear, bulkload_size: 0/10000000, [12233108.]/[3480885.]=[3.51436718], model_change: False
index: alex, case:21, dataset:linear, bulkload_size: 0/20000000, [12233108.]/[3414306.]=[3.58289737], model_change: False
index: alex, case:21, dataset:linear, bulkload_size: 0/50000000, [12233108.]/[3365364.]=[3.63500293], model_change: False
index: alex, case:21, dataset:linear, bulkload_size: 0/100000000, [12233108.]/[3966915.]=[3.08378375], model_change: False
index: alex, case:21, dataset:linear, bulkload_size: 1000000/10000000, [4407167.5]/[3480885.]=[1.26610546], model_change: False
index: alex, case

In [2]:
# overfit
import pandas as pd

# case = 21
# index = "alex"

bulkload_sizes = [0, 1000000, 2000000, 5000000, 10000000, 20000000, 50000000, 100000000]
sizes2ratios = {
    0: "0.0",
    1000000: "0.005",
    2000000: "0.01",
    5000000: "0.025",
    10000000: "0.05",
    20000000: "0.1",
    50000000: "0.25",
    100000000: "0.5",
}
# bulkload_ratios = ["0.0", "0.005", "0.01", "0.025", "0.05", "0.1", "0.25", "0.5"]
datasets = ["linear", "covid", "fb-1", "osm"]

df = pd.read_csv("../data/AIO_trimed.csv")
df = df[(df["read_ratio"] != 0)]
df['key_path'] = df['key_path'].apply(lambda x: x.replace('fb', 'fb-1'))
df_grouped = df.groupby(['index_type', 'key_path', 'test_suite', 'init_table_size'])['throughput'].mean().reset_index()
# print(len(df_grouped))
df_grouped.to_csv("grouped.csv", index=False)

# print(df_grouped)

for index in ["alex", "lipp"]:
    for case in [21, 22, 41, 42]:
        # for each bulkload sizes
        for dataset in datasets:
            for bulkload_size in bulkload_sizes:
                # get case throughput
                d1 = df_grouped[(df_grouped["key_path"] == f"datasets/{dataset}") & (df_grouped["init_table_size"] == bulkload_size) & (df_grouped["test_suite"] == case) & (df_grouped["index_type"] == index)]["throughput"].values
                if (case == 41 or case == 42) and bulkload_size == 0:
                    continue
                root_model_file = f"../log/root_smo_profiling/{dataset}/{case}/{sizes2ratios[bulkload_size]}/{index}_insert_root.log"
                d2 = pd.read_csv(root_model_file)
                is_model_change = False if len(d2) == 1 else True
                if is_model_change == True:
                    print(f"index: {index}, case:{case}, dataset:{dataset}, bulkload_size: {bulkload_size}, {case}: {d1}, model_change: {is_model_change}")



index: alex, case:21, dataset:covid, bulkload_size: 0, 21: [7190905.5], model_change: True
index: alex, case:21, dataset:fb-1, bulkload_size: 0, 21: [1618825.], model_change: True
index: alex, case:21, dataset:osm, bulkload_size: 0, 21: [1949739.], model_change: True
index: alex, case:41, dataset:covid, bulkload_size: 1000000, 41: [5435834.], model_change: True
index: alex, case:41, dataset:fb-1, bulkload_size: 1000000, 41: [1673376.], model_change: True
index: alex, case:41, dataset:fb-1, bulkload_size: 2000000, 41: [1633475.], model_change: True
index: alex, case:41, dataset:fb-1, bulkload_size: 5000000, 41: [1601943.], model_change: True
index: alex, case:41, dataset:fb-1, bulkload_size: 10000000, 41: [1563023.], model_change: True
index: alex, case:41, dataset:fb-1, bulkload_size: 20000000, 41: [1593392.], model_change: True
index: alex, case:41, dataset:fb-1, bulkload_size: 50000000, 41: [1883800.], model_change: True
index: alex, case:41, dataset:fb-1, bulkload_size: 100000000, 4